In [19]:
# ─────────────────────────────────────────────────────────────────────────────
# Импорт библиотек и задание путей
# ─────────────────────────────────────────────────────────────────────────────
from pathlib import Path
from datetime import timedelta
import pandas as pd, numpy as np, json, warnings, gc
warnings.filterwarnings("ignore")

ROOT = Path("..")                          # корень репо
DATA = ROOT / "data"

SRC_DIR  = DATA / "processed_model_ready"  # исходный датасет (не трогаем)
PARQ_IN  = SRC_DIR / "m5_features_processed.parquet"

DST_DIR  = DATA / "model_ready"            # новая «рабочая» папка
DST_DIR.mkdir(parents=True, exist_ok=True)
PARQ_OUT = DST_DIR / "m5_features_model.parquet"

SPLITS = DST_DIR / "splits_wf"             # индексы train/val
SPLITS.mkdir(parents=True, exist_ok=True)

In [20]:
# ─────────────────────────────────────────────────────────────────────────────
# Загрузка исходного Parquet
# ─────────────────────────────────────────────────────────────────────────────
df = pd.read_parquet(PARQ_IN)
print(f"Loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")

# приведение даты к datetime64
if df["date"].dtype != "datetime64[ns]":
    df["date"] = pd.to_datetime(df["date"])

Loaded: 58,327,370 rows × 48 columns


In [21]:
# ─────────────────────────────────────────────────────────────────────────────
# Формирование целевых столбцов: лог-цена и «сырая» цена через 28 дней
# ─────────────────────────────────────────────────────────────────────────────
H = 28  # горизонт

df["target_raw"] = df.groupby("id")["y_raw"].shift(-H)
df["target_log"] = df.groupby("id")["y_log"].shift(-H)

# удаляем хвост, где нет таргета (последние 28 дней в данных)
df = df.dropna(subset=["target_log"]).reset_index(drop=True)
print("After shift :", df.shape)

After shift : (57473650, 50)


In [22]:
# ─────────────────────────────────────────────────────────────────────────────
# Контроль: убеждаемся, что target_log = y_log(t+28) для каждого SKU
# ─────────────────────────────────────────────────────────────────────────────
aligned = (
    (df["target_log"] - df.groupby("id")["y_log"].shift(-H))
      .abs().fillna(0) < 1e-6
).all()
print("✓ target_log aligns to y_log(+28d):", aligned)
print("✓ NaN in targets:", df[["target_raw","target_log"]].isna().any().any())

✓ target_log aligns to y_log(+28d): True
✓ NaN in targets: False


In [23]:
# ─────────────────────────────────────────────────────────────────────────────
# Списки признаков и сохранение мета-файла
# ─────────────────────────────────────────────────────────────────────────────
EXCLUDE = [
    "target_raw", "target_log",          # целевые
    "y_raw", "y_log",                    # исходные цены
    "sales", "price_max_365",            # потенциальная утечка
    "id", "d", "wm_yr_wk_x", "wm_yr_wk_y", "d_numeric"  # служебные
]

feature_cols = [c for c in df.columns if c not in EXCLUDE]
num_cols = [c for c in feature_cols
            if df[c].dtype in ("float32","float64","int16","int8")]
cat_cols = [c for c in feature_cols if df[c].dtype.name == "category"]

meta = {
    "numerical":   num_cols,
    "categorical": cat_cols,
    "all_features": feature_cols,
    "target_raw":  "target_raw",
    "target_log":  "target_log",
    "horizon_days": H,
}
with open(DST_DIR / "features.json", "w") as fp:
    json.dump(meta, fp, indent=2)
print("✔ features.json saved")

✔ features.json saved


In [24]:
# ─────────────────────────────────────────────────────────────────────────────
# Сохраняем Parquet с таргетами
# ─────────────────────────────────────────────────────────────────────────────
df.to_parquet(PARQ_OUT, index=False)
print("✔ Parquet saved →", PARQ_OUT.relative_to(DATA))

✔ Parquet saved → model_ready/m5_features_model.parquet


In [25]:
# ─────────────────────────────────────────────────────────────────────────────
# Генерация индексов для 3-х walk-forward окон (28 дней каждое)
# ─────────────────────────────────────────────────────────────────────────────
VAL_LAST = pd.Timestamp("2016-02-29")  # крайнее полное окно в данных
for k in range(3):
    val_start = VAL_LAST - timedelta(days=H * k)
    val_end   = val_start + timedelta(days=H)

    val_mask  = (df.date >= val_start) & (df.date < val_end)
    train_idx = df.index[df.date < val_start].astype("int32")
    val_idx   = df.index[val_mask].astype("int32")

    np.save(SPLITS / f"wf_train_idx_k{k}.npy", train_idx)
    np.save(SPLITS / f"wf_val_idx_k{k}.npy"  , val_idx)

    print(f"k={k}: train {len(train_idx):,} | val {len(val_idx):,}  "
          f"({val_start.date()} → {(val_end - timedelta(days=1)).date()})")


k=0: train 56,619,930 | val 853,720  (2016-02-29 → 2016-03-27)
k=1: train 55,766,210 | val 853,720  (2016-02-01 → 2016-02-28)
k=2: train 54,912,490 | val 853,720  (2016-01-04 → 2016-01-31)


In [26]:
# ─────────────────────────────────────────────────────────────────────────────
# Очистка памяти
# ─────────────────────────────────────────────────────────────────────────────
del df; gc.collect()
print("\n✅  model_ready/ содержит Parquet, features.json и splits_wf/")


✅  model_ready/ содержит Parquet, features.json и splits_wf/
